In [1]:
from fpl import Loader, Player, Team, SimpleExpectedPointsCalculator, Optimizer
import heapq

## Dowloading your personal FPL team

To download your own personal FPL team use `Loader.get_my_team(login, password, manager_id, how, filename)`

To do download from the API you need your login details and `manager_id`. To get your `manager_id`:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/) and navigate to the points tab.
2. Your `manager_id` should appear somewhere in the URL. [FPL APIs Explained](https://www.oliverlooney.com/blogs/FPL-APIs-Explained#how-to-use-authenticated-endpoints) has some screenshots of this.

If the download from the API fails, you can manually download the json blob as a local file:
1. Log on to [Fantasy Premier League](https://fantasy.premierleague.com/)
2. View a json blob of your team by typing this URL `https://fantasy.premierleague.com/api/my-team/{manager_id}/` into your browser with the appropriate `manager_id`
3. Save the blob locally for example as `my_team.json`


In [3]:
# team = Loader.get_my_team(login="email", password="password", manager_id=123456, how="api")
team = Loader.get_my_team(
    login="", password="", manager_id=0, how="local", filename="resources/my_team.json"
)
print(team)

---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=366, name='B.Fernandes', position=3, club=14, cost=84)
Player(element=585

## Creating a candidate list

Create a `candidates` list of players you want to transfer in, you can use the `Loader.find_matching_players(search_name, matching_threshold)` function below to help (remember to search by a player's name as it appears on the fpl website)

In [6]:
Loader.find_matching_players(search_name="alexander arnold", threshold=50)

[(Player(element=115, name='Baleba', position=3, club=5, cost=50),
  'Carlos Baleba'),
 (Player(element=223, name='Garner', position=3, club=8, cost=49),
  'James Garner'),
 (Player(element=240, name='Andreas', position=3, club=9, cost=50),
  'Andreas Hoelgebaum Pereira'),
 (Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=74),
  'Trent Alexander-Arnold'),
 (Player(element=642, name='André', position=3, club=20, cost=50),
  'André Trindade da Costa Neto')]

In [7]:
candidates = [
    Player(element=398, name="Gordon", position=3, club=15, cost=77),
    Player(element=339, name="Virgil", position=2, club=12, cost=64),
    Player(element=354, name="Kovačić", position=3, club=13, cost=54),
    Player(element=321, name="Gakpo", position=4, club=12, cost=76),
    Player(element=364, name="Amad", position=3, club=14, cost=56),
]

## Optimize your team

In [8]:
top_three = Optimizer.optimize_team(
    team,
    candidates,
    SimpleExpectedPointsCalculator,
    gameweek=Loader.get_next_gameweek(),
    horizon=5,
    max_transfers=2,
    gamma=0.8,
    wildcard=False,
)
third_best_score, third_best_team = heapq.heappop(top_three)
second_best_score, second_best_team = heapq.heappop(top_three)
first_best_score, first_best_team = heapq.heappop(top_three)
print("Score: {}".format(first_best_score))
print(first_best_team)

Score: 253.45865000000006
---------------------------------------------------------------------------
{GKPS}
Player(element=201, name='Henderson', position=1, club=7, cost=44)
Player(element=521, name='Fabianski', position=1, club=19, cost=41)
---------------------------------------------------------------------------
{DEFS}
Player(element=70, name='Kerkez', position=2, club=3, cost=47)
Player(element=231, name='Mykolenko', position=2, club=8, cost=43)
Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72)
Player(element=399, name='Hall', position=2, club=15, cost=47)
Player(element=475, name='Taylor', position=2, club=17, cost=39)
---------------------------------------------------------------------------
{MIDS}
Player(element=99, name='Mbeumo', position=3, club=4, cost=78)
Player(element=182, name='Palmer', position=3, club=6, cost=109)
Player(element=328, name='M.Salah', position=3, club=12, cost=132)
Player(element=364, name='Amad', position=3, club=14, cost=56)

## Interrogating optimization results

The below could probably be put into functions somewhere

In [7]:
def get_discounted_reward(
    player_id: int, number_gameweeks: int, gamma: float = 1
) -> float:
    """Gets the discounted reward of a candidate over a certain number of gameweeks
    :param player_id: the player which you want the reward for
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor

    :return: discounted reward
    """
    discounted_reward = 0
    discount_factor = 1
    next_gameweek = Loader.get_next_gameweek()
    for gw in range(next_gameweek, next_gameweek + number_gameweeks):
        discounted_reward += (
            discount_factor
            * SimpleExpectedPointsCalculator.get_expected_points(player_id, gw)
        )
        discount_factor *= gamma

    return discounted_reward


def print_sorted_candidates(
    candidates: list[int], number_gameweeks: int, gamma: float = 1
) -> None:
    """Print out the list of players and rewards they get
    :param candidates: list of player ids
    :param number_gameweeks: how many gameweeks do you want to consider
    :param gamma: discount factor
    """
    sorted_candidates = []
    for i in candidates:
        r = get_discounted_reward(i, number_gameweeks, gamma)
        sorted_candidates.append((i, r))

    sorted_candidates = reversed(sorted(sorted_candidates, key=lambda x: x[1]))
    for i, r in sorted_candidates:
        web_name = Loader.get_player_basic_info(i)["web_name"]
        print(f"id: {i}, name: {web_name}, reward: {r}")

    return

In [8]:
print_sorted_candidates(
    candidates_gkps + candidates_defs + candidates_mids + candidates_fwds, 5, 0.80
)

id: 328, name: M.Salah, reward: 41.70952
id: 182, name: Palmer, reward: 26.73724
id: 99, name: Mbeumo, reward: 26.339840000000006
id: 401, name: Isak, reward: 25.94052
id: 311, name: Alexander-Arnold, reward: 23.82696
id: 321, name: Gakpo, reward: 23.390800000000006
id: 447, name: Wood, reward: 21.861120000000003
id: 364, name: Amad, reward: 20.947680000000002
id: 207, name: Mateta, reward: 19.816200000000006
id: 235, name: Pickford, reward: 19.415760000000002
id: 366, name: B.Fernandes, reward: 17.8382
id: 398, name: Gordon, reward: 17.45248
id: 70, name: Kerkez, reward: 16.64648
id: 201, name: Henderson, reward: 15.614200000000004
id: 231, name: Mykolenko, reward: 15.599360000000004
id: 129, name: João Pedro, reward: 15.59648
id: 399, name: Hall, reward: 14.931280000000001
id: 585, name: I.Sarr, reward: 14.68976
id: 521, name: Fabianski, reward: 8.77688
id: 220, name: Calvert-Lewin, reward: 6.654760000000001
id: 475, name: Taylor, reward: 3.1783599999999996


In [14]:
Optimizer.optimal_formation(first_best_team, SimpleExpectedPointsCalculator, 24)

{'gkps': frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}),
 'defs': frozenset({Player(element=231, name='Mykolenko', position=2, club=8, cost=43),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=99, name='Mbeumo', position=3, club=4, cost=78),
            Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=364, name='Amad', position=3, club=14, cost=56)}),
 'fwds': frozenset({Player(element=321, name='Gakpo', position=4, club=12, cost=76),
            Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=328, name='M.Salah', position=3, club=12, cost=132),
 'total_exp_point

In [15]:
Optimizer.optimal_formation(second_best_team, SimpleExpectedPointsCalculator, 24)

{'gkps': frozenset({Player(element=235, name='Pickford', position=1, club=8, cost=51)}),
 'defs': frozenset({Player(element=231, name='Mykolenko', position=2, club=8, cost=43),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=99, name='Mbeumo', position=3, club=4, cost=78),
            Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=364, name='Amad', position=3, club=14, cost=56),
            Player(element=585, name='I.Sarr', position=3, club=7, cost=57)}),
 'fwds': frozenset({Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=328, name='M.Salah', position=3, club=12, cost=132),
 'total_exp_points

In [16]:
Optimizer.optimal_formation(team, SimpleExpectedPointsCalculator, 24)

{'gkps': frozenset({Player(element=201, name='Henderson', position=1, club=7, cost=44)}),
 'defs': frozenset({Player(element=231, name='Mykolenko', position=2, club=8, cost=43),
            Player(element=311, name='Alexander-Arnold', position=2, club=12, cost=72),
            Player(element=399, name='Hall', position=2, club=15, cost=47)}),
 'mids': frozenset({Player(element=99, name='Mbeumo', position=3, club=4, cost=78),
            Player(element=182, name='Palmer', position=3, club=6, cost=109),
            Player(element=328, name='M.Salah', position=3, club=12, cost=132),
            Player(element=366, name='B.Fernandes', position=3, club=14, cost=84),
            Player(element=585, name='I.Sarr', position=3, club=7, cost=57)}),
 'fwds': frozenset({Player(element=401, name='Isak', position=4, club=15, cost=92),
            Player(element=447, name='Wood', position=4, club=16, cost=70)}),
 'captain': Player(element=328, name='M.Salah', position=3, club=12, cost=132),
 'total_ex